# INTRODUCTION

# Gold project

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date,datetime,timedelta
from plotly import graph_objs as go
%matplotlib inline

In [ ]:
Daily = pd.read_csv('daily_gold_rate.csv')
Annual = pd.read_csv('annual_gold_rate.csv')

In [ ]:
Daily.info()

In [ ]:
Daily.describe()

In [ ]:
Daily.head

In [ ]:
Daily.isnull()

In [ ]:
Daily['CNY'].fillna(value= Daily['CNY'].mean(), inplace = True)

In [ ]:
Daily['Date']= pd.to_datetime(Daily['Date'],format ="%Y-%m-%d")

In [ ]:
Daily

In [ ]:
def plot_graph(country):
    six_step = dict(count=6,label="6M",step="month",stepmode="backward")
    ytd_dict = dict(count=1,label="YTD",step="year",stepmode="todate")
    year1_dict = dict(count=1,label="1Y",step="year",stepmode="backward")
    year5_dict = dict(count=5,label="5Y",step="year",stepmode="backward")
    all_dict = dict(label='MAX',step="all")
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=Daily['Date'], y=Daily[f'{country}'],mode='lines'))
    fig.layout.update(title_text=f"Price of gold throughout the years in {country}")
    fig.update_layout(xaxis=dict(rangeselector=dict(buttons=list([six_step,ytd_dict,year1_dict,year5_dict,all_dict])),rangeslider=dict(visible=True),type="date"))
    fig.update_layout(yaxis_title=f'Price in {country}')
    fig.show()

In [ ]:
currencies = ['USD','EUR','GBP','INR','AED','CNY']
for i in currencies:
    plot_graph(i)
plt.savefig("Price of gold throughout the years.pdf")
plt.show()

In [ ]:
Annual.info()

In [ ]:
fcrisis = Annual[(Annual['Date']>=2006) & (Annual['Date']<=2010)]
fcrisis['Date'] = pd.to_datetime(fcrisis['Date'], format="%Y")
fcrisis['Date'] = pd.DatetimeIndex(fcrisis['Date']).year

In [ ]:
fcrisis.head()

 # GOLD RATES AROUND THE YEARS OF 2008'S GLOBAL ECONOMIC CRISIS

In [ ]:
# CREATING PERCENTAGE CHANGE COLUMN
for currency in currencies:
    fcrisis[f'{currency}_pct_change'] = fcrisis[f'{currency}'].pct_change()
    fcrisis[f'{currency}_pct_change'] = fcrisis[f'{currency}_pct_change'].transform(lambda x: x*100)
    fcrisis[f'{currency}_pct_change'] = fcrisis[f'{currency}_pct_change'].fillna(0)


In [ ]:
from plotly.subplots import make_subplots
def crisis_viz(data,currency,crisis):
    fig = make_subplots(rows=1, cols=2,subplot_titles=(f"Price of gold in {currency}","Percentage change in gold price"))
    fig.add_trace(go.Bar(x=data['Date'], y=data[f'{currency}'],width=0.6),row=1,col=1)
    fig.add_trace(go.Bar(x=data['Date'],y=data[f'{currency}_pct_change']),row=1,col=2)
    # Update xaxis properties
    fig.update_xaxes(title_text="Years")
    # Update yaxis properties
    fig.update_yaxes(title_text=f"Price of gold in {currency}", row=1, col=1)
    fig.update_yaxes(title_text="Percentage change in rate of gold", row=1, col=2)
    # Update title and height
    fig.update_layout(title_text=f'Price and Precentage change in {currency} during {crisis}',showlegend=False)
    fig.show()

for i in currencies:
    crisis_viz(fcrisis,i,'Global economic crisis')
    
plt.tight_layout()

# EFFECT OF COVID 19 ON GOLD RATES

In [ ]:
pandemic = Annual[(Annual['Date']>=2018) & (Annual['Date']<=2021)]
pandemic['Date'] = pd.to_datetime(pandemic['Date'], format="%Y")
pandemic['Date'] = pd.DatetimeIndex(pandemic['Date']).year

# CREATING PERCENTAGE CHANGE COLUMN
for currency in currencies:
    pandemic[f'{currency}_pct_change'] = pandemic[f'{currency}'].pct_change()
    pandemic[f'{currency}_pct_change'] = pandemic[f'{currency}_pct_change'].transform(lambda x: x*100)
    pandemic[f'{currency}_pct_change'] = pandemic[f'{currency}_pct_change'].fillna(0)

In [ ]:
pandemic.info()

In [ ]:
for i in currencies:
    crisis_viz(pandemic,i,'COVID 19 Pandemic')

# MONTHS THAT GOLD RATE HIT IT'S LOWEST IN THE YEAR(EXAMING THIS TREND FOR 2014 - 2021)

In [ ]:
daily_after2014 = Daily[(Daily['Date'] >= "2014-01-01") & (Daily['Date'] <= "2021-01-01")]

In [ ]:
import calendar
def get_min(year,country):
    data_year = daily_after2014[daily_after2014['Date'].dt.year == year ]
    return data_year[data_year[f'{country}']==data_year[f'{country}'].min()]['Date'].iloc[0].month
ar = [2014,2015,2016,2017,2018,2019,2020,2021]
months=[]

for k in currencies:
    i=0
    while i<len(ar):
        m = get_min(ar[i],k)
        name = calendar.month_name[m]
        i+=1
        months.append(name)
chunk_size = len(ar)
chunked_list = [months[i:i+chunk_size] for i in range(0, len(months), chunk_size)]

In [ ]:
def min_viz(i,currency):
    fig = go.Figure([go.Bar(x=ar,y=chunked_list[i],width=0.6,marker_color="rgb(255, 166, 0)")])
    fig.update_xaxes(title_text="Years")
    fig.update_yaxes(title_text="Months")
    fig.update_yaxes(categoryorder='array', categoryarray= [' ','January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'])
    fig.update_layout(title_text=f'In which month was the gold rate lowest in {currency}')
    fig.show()

In [ ]:
for i in range(0,len(currencies)):
    min_viz(i,currencies[i])